# PDF Support

This notebook demonstrates PDF processing support for both `Call` and `Chat` objects. PDFs can be provided as URLs or local file paths with various parsing engines.

In [1]:
from irouter import Call, Chat
from IPython.display import Markdown, display

# To load OPENROUTER_API_KEY from .env file create a .env file at the root of the project with OPENROUTER_API_KEY=your_api_key
# Alternatively pass api_key=your_api_key to the Call or Chat class
from dotenv import load_dotenv

load_dotenv()

True

For this example we use Moonshot AI's `kimi-k2` model.

If the selected LLM has native file processing capabilities, that parser be used. Else the `mistral-ocr` parser is used, which has some small costs associated with it.

Under the `PDF Parsing Configuration` section in this notebook you can see how to configure a (free) PDF parsing engine. For more details on PDF support in OpenRouter and pricing, check [this docs page](https://openrouter.ai/docs/features/images-and-pdfs#pdf-support).

To see an overview of which LLMs support file input, check the [OpenRouter Model Overview](https://openrouter.ai/models?fmt=cards&input_modalities=audio%2Cfile).

In [2]:
model = "moonshotai/kimi-k2"
# The "Attention Is All You Need" paper
pdf_url = "https://proceedings.neurips.cc/paper_files/paper/2017/file/3f5ee243547dee91fbd053c1c4a845aa-Paper.pdf"

In this example we will ask the LLM questions about the "Attention Is All You Need" paper.

<img src="https://nlp.seas.harvard.edu/images/the-annotated-transformer_0_0.png" width="300"/>



# PDF URL

The simplest way to work with PDF files in `irouter` is to pass the URL of the PDF file and instruction as a list of strings.


In [3]:
c = Call(model)

In [4]:
display(Markdown((c([pdf_url, "What is the main contribution of this paper?"]))))

The **main contribution** of the paper *"Attention Is All You Need"* is the introduction of the **Transformer**, a **new neural network architecture** that **dispenses with recurrence and convolutions entirely**, instead relying **solely on self-attention mechanisms** to model sequential data.  

Key aspects:  
1. **Pure attention-based model**: The Transformer replaces recurrent (RNNs, LSTM) and convolutional layers with **multi-head self-attention**, enabling **better parallelization and faster training**.  
2. **Superior performance**: It achieves **state-of-the-art results** on **English-German (28.4 BLEU)** and **English-French (41.0 BLEU)** WMT 2014 translation tasks.  
3. **Computational efficiency**: Reduces training time significantly (e.g., 3.5 days on 8 GPUs) compared to prior models.  
4. **Global dependencies**: Self-attention allows **constant-time operations** for long-range dependencies, unlike sequential RNNs.  

This marks a paradigm shift from recurrence to **attention-only** sequence modeling.

# PDF Parsing configuration

You can specify different PDF parsing engines using the `extra_body` parameter. For example, use the `pdf-text` engine for free parsing. Check [this docs page](https://openrouter.ai/docs/features/multimodal/pdfs#plugin-configuration) for more details on plugin configuration.

In [5]:
extra_body = {"plugins": [{"id": "file-parser", "pdf": {"engine": "pdf-text"}}]}

In [6]:
display(
    Markdown(
        c(
            [pdf_url, "Summarize the key innovations in this paper."],
            extra_body=extra_body,
        )
    )
)

Key innovations introduced in “Attention Is All You Need”  
• Transformer architecture: First sequence-to-sequence model built solely on attention, completely replacing recurrent or convolutional layers with stacked multi-head self-attention and feed-forward sub-layers.  
• Multi-Head Self-Attention: Decomposes attention into h=8 parallel “heads”, each operating on lower-dimensional key/query/value projections, giving constant-time path length between any two positions and enabling highly parallel computation.  
• Scaled Dot-Product Attention: Adds 1/√dk scaling to vanilla dot-product attention to stabilize gradients with large key dimensions (dk).  
• Positional Encodings: Sinusoidal fixed-frequency encodings inject token-order information without recurrence or convolution and allow length extrapolation.  
• Residual connections + Layer Normalization around each sub-layer ensure stable optimization of deep stacks (N=6 encoder & decoder layers).  
• Masked Decoder Self-Attention: Uses a causal mask so decoding remains auto-regressive while all positions can attend to earlier ones in parallel.  
• Efficiency: Eliminates sequential dependencies within each layer, making training far more parallelizable; achieves new state-of-the-art BLEU scores (28.4 EN-DE, 41.0 EN-FR) in a fraction of the training cost of previous ensemble models.

# Chat with PDF

In contrast to the `Call` class, the `Chat` tracks history and token usage.

In [7]:
chat = Chat(model)

In [8]:
display(Markdown(chat([pdf_url, "What is this paper about?"])))

This is the landmark paper “Attention Is All You Need,” which introduces the **Transformer** architecture.  In a nutshell:

- **Core idea**: We no longer need recurrent or convolutional layers to model sequences; instead, we can achieve state-of-the-art results with *attention mechanisms alone*.

- **Key contribution**:  
  – A new, fully attention-based encoder–decoder model  
  – Multi-head self-attention as the main work-horse  
  – Sinusoidal/positional encodings to inject sequence-order information

- **Results**:  
  – On WMT 2014 English–German translation: 28.4 BLEU (best single model, >2 BLEU over previous best).  
  – On English–French: 41.0 BLEU (new single-model SOTA) at ~1/4 of the prior training cost.

- **Impact**: Opens the door to far greater parallelism during training, dramatically faster convergence, and subsequent breakthroughs in language modeling (BERT, GPT, etc.) and beyond.

Now we can ask follow-up questions about the PDF. `Chat` will update history and token usage.

In [9]:
display(Markdown(chat("What are the key advantages of this approach over RNNs?")))

The Transformer (purely-attention) design yields several clear wins over recurrent models:

1. **Massive parallelization**  
   RNNs are inherently sequential (compute *h*\_{t} after *h*\_{t-1}); the Transformer can process all positions in parallel, leading to much better GPU utilization and shorter training time.

2. **Constant-time path length between any pair of positions**  
   Self-attention lets every position directly attend to every other position in a *constant* number of layer operations, whereas an RNN needs *O(n)* operations to propagate information from the first to the last token. Shorter gradient paths ease learning of long-range dependencies.

3. **Better scaling to longer sequences**  
   Because the number of operations that relate distant positions does *not* increase with sequence length beyond the quadratic attention itself, the model enjoys more predictable behavior than stacked RNNs as sequences grow.

4. **Higher training efficiency / less wall-clock time**  
   Empirically, the base Transformer reaches state-of-the-art translation quality in ~12 hours on 8 P100s—far faster than comparable RNN systems reported in the literature, which often train for weeks.

5. **Interpretable inductive bias**  
   Attention weights themselves provide a direct, interpretable view of what the model “paid attention to” at each step (individual heads often correspond to syntactic or semantic relations).

In [10]:
chat.history

[{'role': 'system', 'content': 'You are a helpful assistant.'},
 {'role': 'user',
  'content': [{'type': 'file',
    'file': {'filename': 'document.pdf',
     'file_data': 'https://proceedings.neurips.cc/paper_files/paper/2017/file/3f5ee243547dee91fbd053c1c4a845aa-Paper.pdf'}},
   {'type': 'text', 'text': 'What is this paper about?'}]},
 {'role': 'assistant',
  'content': 'This is the landmark paper “Attention Is All You Need,” which introduces the **Transformer** architecture.  In a nutshell:\n\n- **Core idea**: We no longer need recurrent or convolutional layers to model sequences; instead, we can achieve state-of-the-art results with *attention mechanisms alone*.\n\n- **Key contribution**:  \n  – A new, fully attention-based encoder–decoder model  \n  – Multi-head self-attention as the main work-horse  \n  – Sinusoidal/positional encodings to inject sequence-order information\n\n- **Results**:  \n  – On WMT 2014 English–German translation: 28.4 BLEU (best single model, >2 BLEU over 

In [11]:
chat.usage

{'prompt_tokens': 18738, 'completion_tokens': 502, 'total_tokens': 19240}